In [1]:
import sys
import cudaq

In [2]:
@cudaq.kernel
def kernel(qubit_count: int):
    qubits = cudaq.qvector(qubit_count)
    h(qubits[0])
    for i in range(1, qubit_count):
        x.ctrl(qubits[0], qubits[i])
    mz(qubits)


@cudaq.kernel
def qubit_kernel(qubit_count: int):
    # Allocate our qubits.
    qvector = cudaq.qvector(qubit_count)
    # Place the first qubit in the superposition state.
    h(qvector[0])
    # Loop through the allocated qubits and apply controlled-X,
    # or CNOT, operations between them.
    for qubit in range(qubit_count - 1):
        x.ctrl(qvector[qubit], qvector[qubit + 1])
    # Measure the qubits.
    mz(qvector)

In [3]:
print(f"Running on target {cudaq.get_target().name}")

result = cudaq.sample(kernel, 2)
print(result)  # Example: { 11:500 00:500 }

print(f"test with 20 qubits")
result = cudaq.sample(qubit_kernel, 20)
print(result)  # Example: { 11:500 00:500 }

Running on target nvidia
{ 00:513 11:487 }

test with 20 qubits
{ 00000000000000000000:490 11111111111111111111:510 }



## Sample

In [4]:
qubit_count = 4
print(cudaq.draw(qubit_kernel, qubit_count))
results = cudaq.sample(qubit_kernel, qubit_count)

# Should see a roughly 50/50 distribution between the |00> and
# |11> states. Example: {00: 505  11: 495}

print("Measurement distribution:" + str(results))

     ╭───╮               
q0 : ┤ h ├──●────────────
     ╰───╯╭─┴─╮          
q1 : ─────┤ x ├──●───────
          ╰───╯╭─┴─╮     
q2 : ──────────┤ x ├──●──
               ╰───╯╭─┴─╮
q3 : ───────────────┤ x ├
                    ╰───╯

Measurement distribution:{ 0000:490 1111:510 }



By default, sample produces an ensemble of 1000 shots. This can be changed by specifying an integer argument for the shots_count. Note that there is a subtle difference between how sample is executed with the target device set to a simulator or with the target device set to a QPU. When run on a simulator, the quantum state is built once and then sampled repeatedly, where the number of samples is defined by shots_count. When executed on quantum hardware, the quantum state collapses upon measurement and hence needs to be rebuilt every time to collect a sample.



In [5]:
results = cudaq.sample(qubit_kernel, qubit_count, shots_count=10000)
print("Measurement distribution:" + str(results))

Measurement distribution:{ 0000:4995 1111:5005 }



A variety of methods can be used to extract useful information from a SampleResult. For example, to return the most probable measurement and its respective probability:

In [6]:
most_probable_result = results.most_probable()
probability = results.probability(most_probable_result)
print("Most probable result: " + most_probable_result)
print("Measured with probability " + str(probability), end='\n\n')

Most probable result: 1111
Measured with probability 0.5005



## Async Execution

Asynchronous execution allows to easily parallelize execution of multiple kernels on a multi-processor platform. Such a platform is available, for example, by choosing the target nvidia-mqpu:

In [7]:
@cudaq.kernel
def kernel2(qubit_count: int):
    qvector = cudaq.qvector(qubit_count)
    h(qvector)
    mz(qvector)

num_gpus = cudaq.num_available_gpus()
qubit_count = 3

if num_gpus > 1:
    # Set the target to include multiple virtual QPUs.
    cudaq.set_target("nvidia", option="mqpu")
    # Asynchronous execution on multiple virtual QPUs, each simulated by an NVIDIA GPU.
    result_1 = cudaq.sample_async(kernel,
                                  qubit_count,
                                  shots_count=1000,
                                  qpu_id=0)
    result_2 = cudaq.sample_async(kernel2,
                                  qubit_count,
                                  shots_count=1000,
                                  qpu_id=1)
else:
    # Schedule for execution on the same virtual QPU.
    result_1 = cudaq.sample_async(kernel,
                                  qubit_count,
                                  shots_count=1000,
                                  qpu_id=0)
    result_2 = cudaq.sample_async(kernel2,
                                  qubit_count,
                                  shots_count=1000,
                                  qpu_id=0)

print("Measurement distribution for kernel:" + str(result_1.get()))
print("Measurement distribution for kernel2:" + str(result_2.get()))

Measurement distribution for kernel:{ 000:513 111:487 }

Measurement distribution for kernel2:{ 000:126 001:123 010:131 011:126 100:105 101:141 110:128 111:120 }



## Observe

The cudaq.observe() method takes a kernel and its arguments as inputs, along with a cudaq.SpinOperator.

Using the cudaq.spin module, operators may be defined as a linear combination of Pauli strings. Functions, such as cudaq.spin.i(), cudaq.spin.x(), cudaq.spin.y(), cudaq.spin.z() may be used to construct more complex spin Hamiltonians on multiple qubits.

In [14]:
import cudaq
from cudaq import spin

operator = spin.z(0)
print(operator)  # prints: [1+0j] Z

@cudaq.kernel
def new_kernel():
    qubit = cudaq.qubit()
    h(qubit)

[1+0j] Z



In [15]:
result = cudaq.observe(new_kernel, operator)
print(result.expectation())  # prints: 0.0

result = cudaq.observe(new_kernel, operator, shots_count=1000)
print(result.expectation())  # prints non-zero value

0.0
0.016000000000000014


### Running on a GPU

In [16]:
import sys
import timeit

# Will time the execution of our sample call.
code_to_time = 'cudaq.sample(qubit_kernel, qubit_count, shots_count=1000000)'
qubit_count = 25

# Execute on CPU backend.
cudaq.set_target('qpp-cpu')
print('CPU time')
print(timeit.timeit(stmt=code_to_time, globals=globals(), number=1))

if cudaq.num_available_gpus() > 0:
    # Execute on GPU backend.
    cudaq.set_target('nvidia')
    print('GPU time')  # Example: 0.773286 s.
    print(timeit.timeit(stmt=code_to_time, globals=globals(), number=1))

CPU time
40.480660477885976
GPU time
0.8497714218683541
